<H1> Разработка инструмента для прогнозирования популярности постов в социальных сетях с применением методов машинного обучения </H1>

Проект содержит следующие этапы:
* Сбор БД и визуализация данных
* Токенизация, лемматизация и стемминг данных
* Векторизация
* Классификация
* **Подбор гиперпараметров и демонстрация итоговых результатов**

Для корректной работы кода необходимо загрузить файлы "ITMO_2.csv", "ITMO_2_vectorized_tf_idf
.npz",  полученные в рамках предыдущего шага проекта.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
# Загрузка данных из файла .npz
data_npz = np.load('ITMO_2_vectorized_tf_idf.npz', allow_pickle=True)

# Извлечение переменной 'data' из файла .npz
X_data = data_npz['X']

# Загрузка целевой переменной из файла ITMO_2.csv
y_data = pd.read_csv('ITMO_2.csv')['popular']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# **Подбор гиперпараметров**

## GridSearch
GreedSearch (Жадный Поиск) - это эвристический метод оптимизации, основная идея которого заключается в том, что на каждом шаге алгоритм выбирает локально оптимальное решение, надеясь, что в итоге это приведет к глобальному оптимуму.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
# определяем классификатор
pipeline = Pipeline([('clf_rfc', RandomForestClassifier())])

In [ ]:
# сетка параметров
parameters = {'clf_rfc__n_estimators':[100, 150, 200, 250, 50],
              'clf_rfc__criterion': ['gini', 'entropy', 'log_loss'],
              'clf_rfc__max_depth':[50, 100, 150, 200, 250]
              }

grid_search = GridSearchCV(
    estimator = pipeline,
    param_grid = parameters,
    cv = 3,
    )

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
# лучшие параметры классификатора
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
  print(f'{param_name}: {best_parameters[param_name]}')

In [ ]:
# тестирование
y_pred = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred, digits = 4))

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1: ', f1_score(y_test, y_pred, average='macro'))

## RandomizedSearch
RandomizedSearch (Случайный поиск) - метод оптимизации, который проводит случайную выборку комбинаций гиперпараметров из заданного пространства поиска. Случайный выбор комбинаций дает возможность обнаружить неочевидные, но эффективные настройки

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
pipeline = Pipeline([('clf', RandomForestClassifier())])

In [ ]:
# сетка параметров
parameters = {'clf_rfc__n_estimators':[100, 150, 200, 250, 50],
              'clf_rfc__criterion': ['gini', 'entropy', 'log_loss'],
              'clf_rfc__max_depth':[50, 100, 150, 200, 250]
              }

random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = parameters,
    n_iter = 3,
    random_state = 2023,
    cv = 3,
    )

In [ ]:
%%time
random_search.fit(X_train, y_train)

In [ ]:
# лучшие параметры классификатора
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
  print(f'{param_name}: {best_parameters[param_name]}')

In [ ]:
# тестирование
y_pred = random_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred, digits = 4))

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1: ', f1_score(y_test, y_pred, average='macro'))

## Halving GridSearch
Halving GridSearch (Последовательное сокращение вдвое) - метод оптимизации, при котором последовательно уменьшается количество рассматриваемых комбинаций гиперпараметров на каждом этапе. Подход позволяет эффективно фокусироваться на наиболее перспективных областях пространства гиперпараметров, отбрасывая менее перспективные варианты

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
pipeline = Pipeline([('clf', RandomForestClassifier())])

In [ ]:
# сетка параметров
parameters = {'clf_rfc__n_estimators':[100, 150, 200, 250, 50],
              'clf_rfc__criterion': ['gini', 'entropy', 'log_loss'],
              'clf_rfc__max_depth':[50, 100, 150, 200, 250]
              }

random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions = parameters,
    n_iter = 3,
    random_state = 2023,
    cv = 3,
    )

In [ ]:
%%time

halving_search.fit(X_train, y_train)

In [ ]:
best_parameters = halving_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

In [ ]:
y_pred = halving_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1: ', f1_score(y_test, y_pred, average='macro'))